In [40]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression,Ridge,ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, r2_score
from datetime import datetime
import pandas as pd
import numpy as np
import pygal

In [41]:
# Función que regresa el IV de la variable provista
def IV(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux["IV"] = (aux["%no_evento"] - aux["%evento"])*aux["WOE"]
    return aux["IV"].sum()

In [42]:
df = pd.read_csv("../data/OnlineNewsPopularity.csv").sample(frac = 0.1).reset_index(drop = True)

In [43]:
df.columns = [x.strip() for x in df.columns]

In [44]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/08/10/worst-summer-jobs/,516.0,11.0,367.0,0.545455,1.0,0.722222,7.0,3.0,1.0,...,0.033333,0.8,-0.392469,-0.7,-0.155556,0.3,-0.5,0.2,0.5,1800
1,http://mashable.com/2014/08/22/the-killing-of-...,138.0,13.0,569.0,0.495575,1.0,0.674923,10.0,8.0,0.0,...,0.100000,0.8,-0.276562,-1.0,-0.100000,0.4,0.1,0.1,0.1,1400
2,http://mashable.com/2013/02/26/google-worried-...,681.0,9.0,228.0,0.575221,1.0,0.731343,1.0,0.0,1.0,...,0.050000,0.7,-0.325000,-0.4,-0.250000,1.0,0.3,0.5,0.3,2100
3,http://mashable.com/2014/09/07/civilian-ukrain...,122.0,9.0,363.0,0.527933,1.0,0.750000,8.0,6.0,3.0,...,0.100000,0.5,-0.138889,-0.2,-0.100000,0.0,0.0,0.5,0.0,1000
4,http://mashable.com/2014/07/06/emotional-intel...,186.0,13.0,325.0,0.566154,1.0,0.705000,9.0,7.0,1.0,...,0.062500,0.6,-0.288889,-0.5,-0.166667,0.0,0.0,0.5,0.0,2800


In [45]:
df["success"] = (df["shares"] > df["shares"].quantile(.5))*1 

In [46]:
ls_cont = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 
           'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 
           'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 
           'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
           'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
           'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 
           'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
           'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
           'max_positive_polarity', 'avg_negative_polarity', 'min_negative_polarity', 
           'max_negative_polarity', 'title_subjectivity', 'title_sentiment_polarity', 
           'abs_title_subjectivity', 'abs_title_sentiment_polarity']

In [47]:
target = "shares"
target_disc = "success"

In [48]:
df.head(3)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,success
0,http://mashable.com/2013/08/10/worst-summer-jobs/,516.0,11.0,367.0,0.545455,1.0,0.722222,7.0,3.0,1.0,...,0.8,-0.392469,-0.7,-0.155556,0.3,-0.5,0.2,0.5,1800,1
1,http://mashable.com/2014/08/22/the-killing-of-...,138.0,13.0,569.0,0.495575,1.0,0.674923,10.0,8.0,0.0,...,0.8,-0.276562,-1.0,-0.100000,0.4,0.1,0.1,0.1,1400,0
2,http://mashable.com/2013/02/26/google-worried-...,681.0,9.0,228.0,0.575221,1.0,0.731343,1.0,0.0,1.0,...,0.7,-0.325000,-0.4,-0.250000,1.0,0.3,0.5,0.3,2100,1


In [49]:
df["date"] = pd.to_datetime(df["url"].str.split("/").str[3:6].str.join("/")) # Obtención de fecha
df["month"] = df["date"].dt.strftime("%Y-%m-01") # Generación de mes

In [50]:
df

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,success,date,month
0,http://mashable.com/2013/08/10/worst-summer-jobs/,516.0,11.0,367.0,0.545455,1.0,0.722222,7.0,3.0,1.0,...,-0.7,-0.155556,0.300000,-0.500000,0.200000,0.500000,1800,1,2013-08-10,2013-08-01
1,http://mashable.com/2014/08/22/the-killing-of-...,138.0,13.0,569.0,0.495575,1.0,0.674923,10.0,8.0,0.0,...,-1.0,-0.100000,0.400000,0.100000,0.100000,0.100000,1400,0,2014-08-22,2014-08-01
2,http://mashable.com/2013/02/26/google-worried-...,681.0,9.0,228.0,0.575221,1.0,0.731343,1.0,0.0,1.0,...,-0.4,-0.250000,1.000000,0.300000,0.500000,0.300000,2100,1,2013-02-26,2013-02-01
3,http://mashable.com/2014/09/07/civilian-ukrain...,122.0,9.0,363.0,0.527933,1.0,0.750000,8.0,6.0,3.0,...,-0.2,-0.100000,0.000000,0.000000,0.500000,0.000000,1000,0,2014-09-07,2014-09-01
4,http://mashable.com/2014/07/06/emotional-intel...,186.0,13.0,325.0,0.566154,1.0,0.705000,9.0,7.0,1.0,...,-0.5,-0.166667,0.000000,0.000000,0.500000,0.000000,2800,1,2014-07-06,2014-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3959,http://mashable.com/2014/01/11/fitness-gadgets...,362.0,11.0,537.0,0.561538,1.0,0.713018,8.0,6.0,8.0,...,-0.8,-0.125000,0.388889,0.127778,0.111111,0.127778,1500,1,2014-01-11,2014-01-01
3960,http://mashable.com/2014/10/14/globe-records-w...,85.0,9.0,922.0,0.508909,1.0,0.671329,28.0,1.0,1.0,...,-0.8,-0.125000,0.400000,-0.400000,0.100000,0.400000,581,0,2014-10-14,2014-10-01
3961,http://mashable.com/2014/08/25/michael-brown-f...,136.0,8.0,80.0,0.759494,1.0,0.933333,3.0,1.0,1.0,...,-0.5,-0.400000,0.500000,0.250000,0.000000,0.250000,634,0,2014-08-25,2014-08-01
3962,http://mashable.com/2014/01/08/gabby-giffords-...,365.0,11.0,143.0,0.678571,1.0,0.744681,4.0,3.0,0.0,...,-0.7,-0.200000,0.100000,-0.150000,0.400000,0.150000,896,0,2014-01-08,2014-01-01


In [51]:
sorted(df["month"].unique())

['2013-01-01',
 '2013-02-01',
 '2013-03-01',
 '2013-04-01',
 '2013-05-01',
 '2013-06-01',
 '2013-07-01',
 '2013-08-01',
 '2013-09-01',
 '2013-10-01',
 '2013-11-01',
 '2013-12-01',
 '2014-01-01',
 '2014-02-01',
 '2014-03-01',
 '2014-04-01',
 '2014-05-01',
 '2014-06-01',
 '2014-07-01',
 '2014-08-01',
 '2014-09-01',
 '2014-10-01',
 '2014-11-01',
 '2014-12-01']

In [52]:
validate = df[df["date"] >= datetime(2014, 6, 1)].reset_index(drop = True).copy() # Generación de validate por fecha
train = df.drop(index = validate.index).reset_index(drop = True).copy()


In [53]:
Xt = train[ls_cont]
yr = train[target]
yc = train[target_disc]

## Modelado

In [54]:
logreg = LogisticRegression()

In [55]:
logreg.fit(Xt, yc)

/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [56]:
ls_score = cross_val_score(estimator=logreg, X = Xt, y=yc, n_jobs=-1, scoring="roc_auc", cv = 4)

In [57]:
ls_score

array([0.65878711, 0.62535057, 0.60423573, 0.6258071 ])

In [58]:
np.mean(ls_score), np.std(ls_score)

(0.6285451280158628, 0.0195142635672131)

In [59]:
validate["yc_hat"] = logreg.predict(validate[ls_cont])

In [20]:
### Regresion

In [60]:
reg = DecisionTreeRegressor()

In [61]:
reg.fit(Xt, yr)

DecisionTreeRegressor()

In [62]:
ls_score = cross_val_score(estimator=reg, X = Xt, y=yr, n_jobs=-1, scoring="r2", cv = 4)

In [63]:
np.mean(ls_score), np.std(ls_score)

(-3.0761511927170133, 2.1004468604478146)

In [64]:
validate["yr_hat"] = reg.predict(validate[ls_cont])

## PSI

In [65]:
dc_bins = {}
for feat in ls_cont:
    dc_bins.update({feat: pd.cut(Xt[feat], bins=3, retbins=True)[1]})

In [66]:
# Cálculo de PSI
for feat in dc_bins.keys():
    for mes in sorted(validate["month"].unique()):
        aux_t = pd.cut(Xt[feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_train"}) # Segmentación de variable continua
        aux_v = pd.cut(validate.loc[validate["month"] == mes, feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_valid"}) # Aplicación de segmentos a validación
        aux = aux_t.merge(aux_v, on = "index")
        aux["diff"] = aux[f"{feat}_valid"] - aux[f"{feat}_train"] # Diferencia en proporciones
        aux["log"] = np.log(aux[f"{feat}_valid"] / aux[f"{feat}_train"]) # Logaritmo del cociente de proporciones
        aux["PSI"] = aux["diff"] * aux["log"] # Cálculo de PSI por rango
        print(f'PSI of {feat} in {mes}: {aux["PSI"].sum()}') # Impresión de PSI total de la variable
    break

PSI of n_tokens_title in 2014-06-01: 0.024268381629367567
PSI of n_tokens_title in 2014-07-01: 0.01999712388729718
PSI of n_tokens_title in 2014-08-01: 0.025254724598932063
PSI of n_tokens_title in 2014-09-01: 0.05794335312827864
PSI of n_tokens_title in 2014-10-01: 0.02236764515584619
PSI of n_tokens_title in 2014-11-01: 0.06876615325443575
PSI of n_tokens_title in 2014-12-01: 0.09328209183524613


## Desempeño de Poder predictivo

In [67]:
for feat in ls_cont:
    for mes in sorted(validate["month"].unique()):
        kb = SelectKBest(k=1, score_func=f_regression) # Medición de poder predictivo
        kb.fit(validate.loc[validate["month"] == mes, [feat]], validate.loc[validate["month"] == mes, "shares"]) # Segmentación por mes
        print(f'Predictive power of {feat} in {mes}: {kb.scores_[0]}') # Muestra de poder predictivo
    break

Predictive power of n_tokens_title in 2014-06-01: 0.378313265853346
Predictive power of n_tokens_title in 2014-07-01: 0.029962096523564227
Predictive power of n_tokens_title in 2014-08-01: 0.41729599559039227
Predictive power of n_tokens_title in 2014-09-01: 0.07589907734141284
Predictive power of n_tokens_title in 2014-10-01: 0.0007577531591167777
Predictive power of n_tokens_title in 2014-11-01: 0.11237611502599533
Predictive power of n_tokens_title in 2014-12-01: 0.061175247389279146


In [69]:
# Prueba de poder predictivo discreta (IV)
for feat in ls_cont:
    for mes in sorted(validate["month"].unique()):
        validate[f"C_{feat}"] = pd.cut(validate[feat], bins=dc_bins[feat])
        print(f'Predictive power of {feat} in {mes}: {IV(validate[validate["month"] == mes], f"C_{feat}", "success")}')
    break

Predictive power of n_tokens_title in 2014-06-01: 0.07615697832227218
Predictive power of n_tokens_title in 2014-07-01: 0.006482477502124616
Predictive power of n_tokens_title in 2014-08-01: 0.14397050971107794
Predictive power of n_tokens_title in 2014-09-01: 0.034426510063114084
Predictive power of n_tokens_title in 2014-10-01: 0.022097139066905712
Predictive power of n_tokens_title in 2014-11-01: 0.028005166425965292
Predictive power of n_tokens_title in 2014-12-01: 0.043658003150634964


## Estabilidad de características


In [70]:
# Estabilidad de características
for feat in dc_bins.keys():
    for mes in sorted(validate["month"].unique()):
        aux_t = pd.cut(Xt[feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_train"})
        aux_v = pd.cut(validate.loc[validate["month"] == mes, feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_valid"})
        aux = aux_t.merge(aux_v, on = "index")
        aux["diff"] = abs(aux[f"{feat}_valid"] - aux[f"{feat}_train"])
        print(f'Stability KS of {feat} in {mes}: {max(aux["diff"])}')
    break
    

Stability KS of n_tokens_title in 2014-06-01: 0.06759816420193776
Stability KS of n_tokens_title in 2014-07-01: 0.04867924528301887
Stability KS of n_tokens_title in 2014-08-01: 0.06446871896722939
Stability KS of n_tokens_title in 2014-09-01: 0.09867924528301886
Stability KS of n_tokens_title in 2014-10-01: 0.06961443806398682
Stability KS of n_tokens_title in 2014-11-01: 0.11463669209152949
Stability KS of n_tokens_title in 2014-12-01: 0.10477680625862862


## Performance

In [71]:
# Evaluación de la precisión del modelo en "producción" (discreto)
for month in sorted(validate["month"].unique()):
    aux = validate[validate["month"] == month]
    
    print(month, roc_auc_score(y_score=aux["yc_hat"], y_true=aux["success"]))

2014-06-01 0.5631578947368422
2014-07-01 0.5992362576625465
2014-08-01 0.6658163265306123
2014-09-01 0.680995475113122
2014-10-01 0.5523775655354604
2014-11-01 0.6312701334560517
2014-12-01 0.5903125


In [72]:
# Evaluación de la precisión del modelo en "producción" (continuo)
for month in sorted(validate["month"].unique()):
    aux = validate[validate["month"] == month]
    print(month, r2_score(y_pred=aux["yr_hat"], y_true=aux["shares"]))

2014-06-01 0.5844418995899179
2014-07-01 -0.1752005353357049
2014-08-01 0.5447577274453037
2014-09-01 0.48331880953933315
2014-10-01 0.24418076348647688
2014-11-01 0.9411348726870984
2014-12-01 0.03479311206436331


In [73]:
from scipy.stats import ks_2samp

In [74]:
logreg.predict_proba(Xt)

array([[0.46221505, 0.53778495],
       [0.45486788, 0.54513212],
       [0.51140327, 0.48859673],
       ...,
       [0.50806088, 0.49193912],
       [0.42133606, 0.57866394],
       [0.5856636 , 0.4143364 ]])

In [76]:
logreg.predict_proba(Xt)[:,1]

array([0.53778495, 0.54513212, 0.48859673, ..., 0.49193912, 0.57866394,
       0.4143364 ])

In [75]:
logreg.predict(Xt)

array([1, 1, 0, ..., 0, 1, 0])

In [77]:
train["proba"] = logreg.predict_proba(Xt)[:,1]
validate["proba"] = logreg.predict_proba(validate[ls_cont])[:,1]
train["pred_clas"] = logreg.predict(Xt)
validate["pred_clas"] = logreg.predict(validate[ls_cont])

In [78]:
aux_train =train.set_index("month")[["proba","pred_clas"]] 
aux_test =validate.set_index("month")[["proba","pred_clas"]] 

In [79]:
aux_train

,proba,pred_clas
month,,
2014-08-01,0.537785,1
2014-04-01,0.545132,1
2013-10-01,0.488597,0
2014-02-01,0.546946,1
2014-05-01,0.445428,0
...,...,...
2014-01-01,0.364661,0
2014-10-01,0.365404,0
2014-08-01,0.491939,0


In [80]:
aux_test

,proba,pred_clas
month,,
2014-08-01,0.437386,0
2014-09-01,0.536683,1
2014-07-01,0.410388,0
2014-08-01,0.351155,0
2014-10-01,0.456340,0
...,...,...
2014-11-01,0.599692,1
2014-06-01,0.399467,0
2014-10-01,0.545907,1


In [81]:
for month in sorted(validate["month"].unique()):
    ks_result = ks_2samp(aux_test.loc[month].query("pred_clas==0")["proba"],
                         aux_test.loc[month].query("pred_clas==1")["proba"])
    print(f"Month {month } KS: {ks_result}")

Month 2014-06-01 KS: KstestResult(statistic=1.0, pvalue=0.0)
Month 2014-07-01 KS: KstestResult(statistic=1.0, pvalue=0.0)
Month 2014-08-01 KS: KstestResult(statistic=1.0, pvalue=3.3306690738754696e-16)
Month 2014-09-01 KS: KstestResult(statistic=1.0, pvalue=1.1102230246251565e-16)
Month 2014-10-01 KS: KstestResult(statistic=1.0, pvalue=0.0)
Month 2014-11-01 KS: KstestResult(statistic=1.0, pvalue=4.440892098500626e-16)
Month 2014-12-01 KS: KstestResult(statistic=1.0, pvalue=8.881784197001252e-16)
